In [3]:
# Install dependencies in Colab
!apt-get update
!apt install -y chromium-chromedriver
!pip install selenium

from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait  # Added this import
from selenium.webdriver.support import expected_conditions as EC  # Added this import
import pandas as pd
import time
import os


Get:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Hit:3 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:4 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [1,381 kB]
Get:6 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:7 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Get:8 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Hit:9 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Get:11 https://r2u.stat.illinois.edu/ubuntu jammy/main all Packages [8,812 kB]
Hit:12 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:13 http://archive.ubuntu.com/ubuntu jammy-upda

In [4]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
import time
import os

In [7]:
# Load your original CSV
df = pd.read_csv("shl_catalog.csv")

# Add new empty columns for enriched data
df["Description"] = ""
df["Job Levels"] = ""
df["Languages"] = ""
df["Assessment Length"] = ""

# Setup Selenium for Google Colab (headless Chrome)
options = Options()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')

driver = webdriver.Chrome(options=options)

In [8]:
# Helper function to extract value based on heading
def extract_field_text(driver, heading_name):
    try:
        blocks = driver.find_elements(By.CSS_SELECTOR, "div.product-catalogue-training-calendar__row.typ")
        for block in blocks:
            try:
                heading = block.find_element(By.TAG_NAME, "h4").text.strip().lower()
                if heading == heading_name.lower():
                    return block.find_element(By.TAG_NAME, "p").text.strip()
            except:
                continue
    except:
        return ""
    return ""

In [9]:
# Loop through each row in the CSV
for i, row in df.iterrows():
    url = row["URL"]
    print(f"Processing ({i+1}/{len(df)}): {url}")
    try:
        driver.get(url)
        time.sleep(2.5)

        df.at[i, "Description"] = extract_field_text(driver, "Description")
        df.at[i, "Job Levels"] = extract_field_text(driver, "Job levels")
        df.at[i, "Languages"] = extract_field_text(driver, "Languages")

        assessment = extract_field_text(driver, "Assessment length")
        if "=" in assessment:
            df.at[i, "Assessment Length"] = assessment.split("=")[-1].strip()
        else:
            df.at[i, "Assessment Length"] = assessment

    except Exception as e:
        print(f" Error at row {i} - URL: {url}")
        print(e)

# Save the enriched version
df.to_csv("shl_catalog_detailed.csv", index=False)
print(" Done! File saved as shl_catalog_detailed.csv")


Processing (1/518): https://www.shl.com/solutions/products/product-catalog/view/account-manager-solution/
Processing (2/518): https://www.shl.com/solutions/products/product-catalog/view/administrative-professional-short-form/
Processing (3/518): https://www.shl.com/solutions/products/product-catalog/view/agency-manager-solution/
Processing (4/518): https://www.shl.com/solutions/products/product-catalog/view/apprentice-8-0-job-focused-assessment-4261/
Processing (5/518): https://www.shl.com/solutions/products/product-catalog/view/apprentice-8-0-job-focused-assessment/
Processing (6/518): https://www.shl.com/solutions/products/product-catalog/view/bank-administrative-assistant-short-form/
Processing (7/518): https://www.shl.com/solutions/products/product-catalog/view/bank-collections-agent-short-form/
Processing (8/518): https://www.shl.com/solutions/products/product-catalog/view/bank-operations-supervisor-short-form/
Processing (9/518): https://www.shl.com/solutions/products/product-cat